    ==============================================================================================

## Answer Bussines questions


### Revenue
- Recap:
    - ord_TotlPaid : total amount paid by the customer for this order (euros)
    - ol_ProdUntPr : the unitary price (euros) of each product at the moment of placing that order
    - ol_ProdQnty :how many units of that product were purchased on that order
- per row(OL) : ol_ProdUntPr * ol_ProdQnty = tmp_ol_price
- group by ORD_ID .sum of the tmp_ol_price = tmp_ord_price
- compare the tmp_ord_price with ord_TotlPaid

In [ ]:
drop_lst =["ord_CreatDate","ol_ProcessDate","PRO_SKU","pro_Name","pro_Desc","pro_InStock","pro_Type"]

In [ ]:
db["tmp_ol_price"] = db.ol_ProdUntPr * db.ol_ProdQnty

In [ ]:
db.drop(columns=drop_lst)

In [ ]:

db_pr=(
    db
    .groupby("ORD_ID")
    .agg(tmp_ord_price=('tmp_ol_price', 'sum'))
    .merge(db,on="ORD_ID")
    .drop(columns=drop_lst)
)

In [ ]:

db_pr= (
    db_pr
    .assign(tmp_ord_Rev = db_pr.ord_TotlPaid-db_pr.tmp_ord_price)
    )

In [ ]:
db_pr.sample()

In [ ]:
db_pr.tmp_diff_ord_Pr.describe()

Calculate outlier

In [ ]:
Q1 = db_pr.tmp_diff_ord_Pr.quantile(0.25)
Q3 = db_pr.tmp_diff_ord_Pr.quantile(0.75)
IQR=Q3-Q1

In [ ]:
db_pr=db_pr.loc[(db_pr.tmp_diff_ord_Pr >= (Q1 - 1.5*IQR)) & (db_pr.tmp_diff_ord_Pr <= (Q3 + 1.5*IQR))]

In [ ]:
db.assign(tmp_discount = db.pro_Pr - db.ol_ProdUntPr)

In [ ]:
db_pr.tmp_discount.describe()

check for orders of 1 orderlines if totalpaid == unit price * quantity

In [ ]:
(db
 .loc[
     (
           (~db.duplicated(subset="ORD_ID")) 
         & (((db.ol_ProdUntPr*db.ol_ProdQnty)>db.ord_TotlPaid*0.9)
         & ((db.ol_ProdUntPr*db.ol_ProdQnty)<db.ord_TotlPaid*1.1))
     )
 ]
)